In [1]:
%cd ..

/home/jhkim/workspace/lrquant


/home/jhkim/workspace/lrquant/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import gc
import os
import typing

import numpy as np
import pandas as pd
import torch

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.style as mplstyle

mplstyle.use("fast")

from IPython.display import display

if "CUDA_VISIBLE_DEVICES" in os.environ:
    print(os.environ["CUDA_VISIBLE_DEVICES"])

torch.set_printoptions(precision=4, sci_mode=False)

0,1


In [ ]:
from models.LMClass import LMClass

lm = LMClass("./huggingface/llama-2-7b-hf")


from datautils import get_loaders

dataloader, _ = get_loaders(
    name="wikitext2",
    nsamples=128,
    seed=2,
    model=lm.model_name,
    seqlen=lm.seqlen,
    cache_dir=f"./cache/wikitext2_128_2/dataloader.cache",
)

In [ ]:
layer_inps = torch.zeros((32, 2048, 4096), dtype=torch.float32)

for i, layer in enumerate(lm.model.model.layers):

    def layer_hook(layer_index):
        def _fn(module, input: tuple[torch.Tensor], output):
            layer_inps[layer_index] = input[0][0].detach().clone()

        return _fn

    layer._forward_hooks.clear()
    layer.register_forward_hook(layer_hook(i))

batch_index = 32
with torch.no_grad():
    lm.model.eval()
    lm.model.to(lm.device)
    lm.model(dataloader[batch_index][0].to(lm.device))
    lm.model.cpu()
    gc.collect()
    torch.cuda.empty_cache()

OutOfMemoryError: CUDA out of memory. Tried to allocate 32.00 MiB. GPU 0 has a total capacity of 47.54 GiB of which 25.06 MiB is free. Process 174438 has 28.76 GiB memory in use. Process 177140 has 15.11 GiB memory in use. Including non-PyTorch memory, this process has 3.62 GiB memory in use. Of the allocated memory 3.37 GiB is allocated by PyTorch, and 1.87 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## 두 가지 종류의 outlier
### outlier token 
- 전체 input sequence에서 유독 높은 vector magnitude들을 가지는 sequence가 있음
- 대부분의 입력 데이터에서 0번 sequence의 token vector magnitude가 큰 값을 가짐
    - <bos> 토큰?
- 적어도 llama-2-7b 모델에서는 하나 이상의 0번이 아닌 outlier token이 존재함
    - 몇 가지 입력 데이터를 확인해본 결과 "."이거나 "\n" 토큰이었음
- 이 outlier token은 여러 outlier component들을 가지지만, 대부분의 components들은 여전히 0에 가까운 값을 가짐
    - 몇 개의 극단적인 outlier component들이 존재

In [ ]:
layer_index = 2
display(pd.DataFrame(layer_inps[layer_index, 0].abs()).sort_values(0, ascending=False).T)
display(pd.DataFrame(layer_inps[layer_index, 30].abs()).sort_values(0, ascending=False).T)

print(repr(lm.tokenizer.decode(dataloader[batch_index][0][0][30])))
print(repr(lm.tokenizer.decode(dataloader[batch_index][0][0][28:32])))
print(repr(lm.tokenizer.decode(dataloader[52][0][0][225:229])))

NameError: name 'layer_inps' is not defined

In [3]:
display(pd.DataFrame(layer_inps[:, 30, (2533, 1415, 1512)].abs(), columns=["2533", "1415", "1512"]))

NameError: name 'layer_inps' is not defined

In [3]:
magnitudes = layer_inps.square().mean(dim=2)  # [32, 2048]
print(magnitudes.shape)
topk = magnitudes.topk(5, dim=1)
display(pd.DataFrame([list(zip(topk.values[i], topk.indices[i])) for i in range(32)]))

NameError: name 'layer_inps' is not defined

### outlier channel
- layer에는 유독 높은 값들을 가지는 channel들이 존재함
    - 여기서 channel은 token vector component의 index를 의미함
        - 더 좋은 표현?
- outlier channel의 대부분의 vector component들이 vector 내에서 이상치로 나타남

In [ ]:
layer_inps_mean = layer_inps.mean(dim=2, keepdim=True)
layer_inps_std = layer_inps.std(dim=2, keepdim=True)
layer_inps_standardized = (layer_inps - layer_inps_mean) / layer_inps_std

def display_channel(channel_index):
    print("Sqared Mean:", layer_inps[layer_index, :, channel_index].square().mean(dim=0), "Standardized Sqared Mean:", layer_inps_standardized[layer_index, :, channel_index].square().mean(dim=0))
    display(pd.DataFrame(layer_inps[layer_index, :, channel_index].abs()).sort_values(0, ascending=False).T)
    
# 전체적으로 outlier인 channel
display_channel(1512)
# display_channel(2298)

# 한두개의 component만 극단적으로 큰 channel
display_channel(2533)
# display_channel(1415)


In [ ]:
df = pd.DataFrame()
df["std_magnitude"] = layer_inps_standardized[layer_index].square().mean(dim=0)
df["magnitude"] = layer_inps[layer_index].square().mean(dim=0)

display(df.sort_values("std_magnitude", ascending=False).T)
display(df.sort_values("magnitude", ascending=False).T)

## Todo
- activation outlier 관련 논문 보기
    - https://arxiv.org/abs/2309.15531
    - https://arxiv.org/abs/2310.08041

- decoder layer의 각 컴포넌트별로 outlier 확인하기
  - 실제로 코딩하려면 decoder 내부 컴포넌트끼리의 연산에 적용시켜야 할 것 같음

- outlier token과 outlier channel을 구분하는 방법